In [1]:
import feature_eng.utils as utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
train, prop = utils.load_train_data()

/home/hzn/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
column_types = list(map(lambda x: prop[x].dtype, prop.columns))

In [4]:
column_types_set = set(column_types)
column_types_set

{dtype('int64'), dtype('float64'), dtype('O')}

In [5]:
column_types_count = list(map(lambda x: (x,column_types.count(x)), column_types_set))
column_types_count

[(dtype('O'), 5), (dtype('float64'), 52), (dtype('int64'), 1)]

In [6]:
object_columns = list(filter(lambda x: prop[x].dtype == 'object', prop.columns))
object_columns

['hashottuborspa',
 'propertycountylandusecode',
 'propertyzoningdesc',
 'fireplaceflag',
 'taxdelinquencyflag']

In [7]:
int_columns = list(filter(lambda x: prop[x].dtype == 'int64', prop.columns))
int_columns

['parcelid']

In [8]:
float_columns = list(filter(lambda x: prop[x].dtype == 'float64', prop.columns))

In [9]:
# object columns
for col in object_columns:
    print(prop[col].describe())
    print("#")
    print(prop[col].value_counts().head())
    print(">>>>>>>>>>>>>>>>>>>>\n") 

count     69014
unique        1
top        True
freq      69014
Name: hashottuborspa, dtype: object
#
True    69014
Name: hashottuborspa, dtype: int64
>>>>>>>>>>>>>>>>>>>>

count     2972940
unique        240
top          0100
freq      1153896
Name: propertycountylandusecode, dtype: object
#
0100    1153896
122      522145
0101     247494
010C     225410
1111     126491
Name: propertycountylandusecode, dtype: int64
>>>>>>>>>>>>>>>>>>>>

count     1978629
unique       5638
top          LAR1
freq       275029
Name: propertyzoningdesc, dtype: object
#
LAR1     275029
LAR3      67105
LARS      54859
LBR1N     52750
LAR2      48808
Name: propertyzoningdesc, dtype: int64
>>>>>>>>>>>>>>>>>>>>

count     5163
unique       1
top       True
freq      5163
Name: fireplaceflag, dtype: object
#
True    5163
Name: fireplaceflag, dtype: int64
>>>>>>>>>>>>>>>>>>>>

count     56462
unique        1
top           Y
freq      56462
Name: taxdelinquencyflag, dtype: object
#
Y    56462
Name: taxdelinquency

In [10]:
# float type columns, many of the columns are actually categorical / boolean
# Supress scientific notation
pd.set_option('display.float_format', lambda x: '%.1f' % x)
for col in float_columns:
    print(prop[col].describe())
    print("#")
    print(prop[col].value_counts().head())
    print(">>>>>>>>>>>>>>>>>>>>\n") 

count   811519.0
mean         1.9
std          3.1
min          1.0
25%          1.0
50%          1.0
75%          1.0
max         13.0
Name: airconditioningtypeid, dtype: float64
#
1.0     742364
13.0     58457
5.0       8795
11.0      1818
12.0        59
Name: airconditioningtypeid, dtype: int64
>>>>>>>>>>>>>>>>>>>>

count   6061.0
mean       7.2
std        2.4
min        2.0
25%        7.0
50%        7.0
75%        7.0
max       27.0
Name: architecturalstyletypeid, dtype: float64
#
7.0     5251
8.0      380
2.0      201
21.0     150
3.0       58
Name: architecturalstyletypeid, dtype: int64
>>>>>>>>>>>>>>>>>>>>

count   1628.0
mean     646.9
std      538.8
min       20.0
25%      272.0
50%      534.0
75%      847.2
max     8516.0
Name: basementsqft, dtype: float64
#
1528.0    25
700.0     21
240.0     21
100.0     18
144.0     16
Name: basementsqft, dtype: int64
>>>>>>>>>>>>>>>>>>>>

count   2973755.0
mean          2.2
std           1.1
min           0.0
25%           2.0
50%        

In [11]:
# correlation between features
# correlation between feature and logerror, abslogerror, positive logerror, negative logerror

# column1 = 'taxamount'
# column2 = 'taxvaluedollarcnt'
# prop.corr()[column1][column2]

# Too many features, kan bu qing
# import seaborn as sns
# sns.heatmap(prop.corr())

corr = prop.corr()

In [12]:
# # Stack approach
# stacked_corr = corr.stack()
# stacked_corr = stacked_corr.reset_index()
# stacked_corr = stacked_corr[stacked_corr['level_0'] != stacked_corr['level_1']]
# # High correlation columns
# threshold = 0.7
# high_corr = stacked_corr[
#     (stacked_corr[0] > threshold) | (stacked_corr[0] < -threshold)]
# high_corr

In [13]:
# https://stackoverflow.com/questions/38552055/how-to-scan-a-panda-dataframe-for-all-values-greater-than-something-and-returns

threshold = 0.7

# Extract values and row, column names
arr = corr.values
index_names = corr.index
col_names = corr.columns

#  Get indices where such threshold is crossed; avoid diagonal elems
R,C = np.where((np.triu(arr,1) > threshold) | (np.triu(arr,1) < -threshold))

# Arrange those in columns and put out as a dataframe
out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
df_out = pd.DataFrame(out_arr,columns=[['row_name','col_name','value']])

/home/hzn/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in greater
/home/hzn/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in less


In [15]:
df_out

,row_name,col_name,value
0,airconditioningtypeid,fips,0.8
1,airconditioningtypeid,rawcensustractandblock,0.8
2,airconditioningtypeid,regionidcounty,-0.8
3,airconditioningtypeid,roomcnt,0.8
4,airconditioningtypeid,censustractandblock,0.8
5,architecturalstyletypeid,unitcnt,-1.0
6,bathroomcnt,bedroomcnt,0.7
7,bathroomcnt,calculatedbathnbr,1.0
8,bathroomcnt,finishedsquarefeet12,0.7
9,bathroomcnt,fullbathcnt,1.0


In [16]:
# # Interesting correlations:

# # bathroom
# 6	bathroomcnt	bedroomcnt	0.7
# 7	bathroomcnt	calculatedbathnbr	1.0 expected
# 9	bathroomcnt	fullbathcnt	1.0 expected
# 13	calculatedbathnbr	fullbathcnt	1.0 expected
# 8	bathroomcnt	finishedsquarefeet12 	0.7 finished living area
# 23	fullbathcnt	finishedsquarefeet12	0.8
# 27  fullbathcnt	finishedsquarefeet15    0.7
# 11	calculatedbathnbr	finishedsquarefeet12	0.8
# 12	calculatedbathnbr	finishedsquarefeet15	0.7 Total area

# # building quality and garage
# 10	buildingqualitytypeid	garagecarcnt	0.8

# # areas
# 14	finishedfloor1squarefeet	calculatedfinishedsquarefeet	0.7
# 15	finishedfloor1squarefeet	finishedsquarefeet12	0.7
# 16	finishedfloor1squarefeet	finishedsquarefeet15	0.8
# 17	finishedfloor1squarefeet	finishedsquarefeet50	1.0
# 18	calculatedfinishedsquarefeet	finishedsquarefeet12	1.0
# 19	calculatedfinishedsquarefeet	finishedsquarefeet13	1.0
# 20	calculatedfinishedsquarefeet	finishedsquarefeet15	1.0
# 21	calculatedfinishedsquarefeet	finishedsquarefeet6	1.0
# 22	finishedsquarefeet12	finishedsquarefeet15	1.0
# 25	finishedsquarefeet13	yardbuildingsqft26	1.0
# 26	finishedsquarefeet15	finishedsquarefeet50	0.8
# 28	finishedsquarefeet15	garagecarcnt	0.7
# 29	finishedsquarefeet15	poolsizesum	0.9
# 30	finishedsquarefeet15	unitcnt	0.8
# 31	finishedsquarefeet15	yardbuildingsqft17	0.8

# # tax
# 24	finishedsquarefeet12	structuretaxvaluedollarcnt	0.7
# 34	structuretaxvaluedollarcnt	taxvaluedollarcnt	0.8
# 35	structuretaxvaluedollarcnt	taxamount	0.8
# 36	taxvaluedollarcnt	landtaxvaluedollarcnt	0.9
# 37	taxvaluedollarcnt	taxamount	0.9
# 38	landtaxvaluedollarcnt	taxamount	0.8